# Autoencoders

* Autoencoders are unsupervised learning models used to learn representations.

* It is composed by two parts: the encoder and the decoder.

* The encoder is responsible for creating a representation of the original input (code, feature embedding) and the decoder is responsible for trying to recreate the original input from this code. Both encoder and decoder can be built from dense layers.

* The decoder output is not the same as the original input as it was transformed by the encoder and not all the information from it is necessarily present in the code. 

* Loss function: as the loss is computed comparing the input to the decoder output, regression related loss functions are used to train the autoencoder. We can use, for example, the mean squared error.

* After training the autoencoder, it is possible to use the representation (code) for a series of applications. It can be used for indexing data, datasearch in large datasets, and classification useing other ML method.

## Undercomplete autoencoders

* An undercomplete autoencoder's encoder obtains representations that have a reduced dimensionality when compared to the input. They can be used to learn a dimensionality reduction.

* A dense autoencoder with a single encoder/decoder layer has mathematical relation to the PCA method: if the data manifold is linear, the autoencoder tends to converge to a projection of the $m$ principal components of the data.

* Hence, this autoencoder makes a compression of the input, with information loss and the code related layer is called "bottloneck".

* The autoencoder learns to recreate, to its best capabilities, the original inputs. So it is trained to learn patterns on the code that relate to a certain feature in the original input.

* If the autoencoder is trained to recreate images of numbers from 0 to 9, for example, and then is feeded with images of clothes, for example, the output will try to find patterns that ressembles a number on the clothing image. Hence, ir recreates clothes images in ressemblance to numbers.

* Deep undercomplete autoencoders: they can have other types of layers like convolutioinal layers and pooling layers. The code layer is ini most cases a fully conected (dense) layer to allow the projection of the data.

## Overcomplete autoencoders

* Intermediary layer (code layer) with bigger or equal dimensionality than the input.

* A simple implementation would learn to "copy" the input, as the code layer has this capacity. Hence, it is necessary to restrain this space.

* To prevent this, we can use regularization or dropout techiniques.

## Denoising autoencoders

* This type of autoencoder, that can be under or overcomplete, is trainet  to remove noise from an input. The input of this neural network corresponds to the original input with some noise added and the loss function is calculated between the final output and the original input.

* This architecture allows the neural network to learn to remove noise from the original input.

* This noise can be introduced by adding some noise (like a gaussian noise) or remove some points (replace by zero) on the original input.